In [1]:
# runs in Chameleon Jupyter environment
git clone --recurse-submodules https://github.com/Ayachisan/MIDAS-iac /work/MIDAS-iac

Cloning into '/work/MIDAS-iac'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 119 (delta 34), reused 112 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 27.37 KiB | 3.91 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Submodule 'ansible/k8s/kubespray' (https://github.com/kubernetes-sigs/kubespray.git) registered for path 'ansible/k8s/kubespray'
Cloning into '/work/MIDAS-iac/ansible/k8s/kubespray'...
remote: Enumerating objects: 81772, done.        
remote: Counting objects: 100% (28/28), done.        
remote: Compressing objects: 100% (19/19), done.        
remote: Total 81772 (delta 20), reused 9 (delta 9), pack-reused 81744 (from 3)        
Receiving objects: 100% (81772/81772), 26.10 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (45837/45837), done.
Submodule path 'ansible/k8s/kubespray': checked out '184b15f8aef4eba40c7433f509b0659b7b66fa44'


In [21]:
cd /work/MIDAS-iac
git branch
git status

* main
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	ansible/.ipynb_checkpoints/
	ansible/ansible.cfg
	ansible/ansible.log
	ansible/k8s/inventory/mycluster/.ipynb_checkpoints/
	ansible/k8s/inventory/mycluster/group_vars/.ipynb_checkpoints/
	k8s/canary/.ipynb_checkpoints/
	k8s/canary/templates/.ipynb_checkpoints/
	k8s/production/templates/.ipynb_checkpoints/
	tf/kvm/.ipynb_checkpoints/
	tf/kvm/.terraform.lock.hcl

nothing added to commit but untracked files present (use "git add" to track)


In [22]:
git fetch origin
git reset --hard origin/main
git submodule update --init --recursive --force

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 461 bytes | 230.00 KiB/s, done.
From https://github.com/Ayachisan/MIDAS-iac
   87985e8..90a1efb  main       -> origin/main
HEAD is now at 90a1efb Update MIDAS submodule content
Submodule path 'ansible/k8s/kubespray': checked out '184b15f8aef4eba40c7433f509b0659b7b66fa44'


### Install and configure Terraform ###

In [2]:
# runs in Chameleon Jupyter environment
mkdir -p /work/.local/bin
wget https://releases.hashicorp.com/terraform/1.10.5/terraform_1.10.5_linux_amd64.zip
unzip -o -q terraform_1.10.5_linux_amd64.zip
mv terraform /work/.local/bin
rm terraform_1.10.5_linux_amd64.zip

--2025-05-11 13:02:47--  https://releases.hashicorp.com/terraform/1.10.5/terraform_1.10.5_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 13.249.59.94, 13.249.59.44, 13.249.59.121, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|13.249.59.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27714924 (26M) [application/zip]
Saving to: ‘terraform_1.10.5_linux_amd64.zip’

terraform_1.10.5_li 100%[===================>]  26.43M  51.6MB/s    in 0.5s    

2025-05-11 13:02:47 (51.6 MB/s) - ‘terraform_1.10.5_linux_amd64.zip’ saved [27714924/27714924]



In [3]:
# runs in Chameleon Jupyter environment
export PATH=/work/.local/bin:$PATH

In [4]:
# runs in Chameleon Jupyter environment
terraform

Usage: terraform [global options] <subcommand> [args]

The available commands for execution are listed below.
The primary workflow commands are given first, followed by
less common or more advanced commands.

Main commands:
  init          Prepare your working directory for other commands
  validate      Check whether the configuration is valid
  plan          Show changes required by the current configuration
  apply         Create or update infrastructure
  destroy       Destroy previously-created infrastructure

All other commands:
  console       Try Terraform expressions at an interactive command prompt
  fmt           Reformat your configuration in the standard style
  force-unlock  Release a stuck lock on the current workspace
  get           Install or upgrade remote Terraform modules
  graph         Generate a Graphviz graph of the steps in an operation
  import        Associate existing infrastructure with a Terraform resource
  login         Obtain and save credentials for a

: 127

Upload the cloud.yaml to terraform directory

In [6]:
# # runs in Chameleon Jupyter environment
cp clouds.yaml /work/MIDAS-iac/tf/kvm/clouds.yaml

### Provision infrastructure with Terraform ###


In [7]:
# runs in Chameleon Jupyter environment
cd /work/MIDAS-iac/tf/kvm

In [8]:
# runs in Chameleon Jupyter environment
export PATH=/work/.local/bin:$PATH

In [9]:
# runs in Chameleon Jupyter environment
unset $(set | grep -o "^OS_[A-Za-z0-9_]*")

Check clouds.yaml have credentials

In [10]:
# runs in Chameleon Jupyter environment
cat clouds.yaml

# This is a clouds.yaml file, which can be used by OpenStack tools as a source
# of configuration on how to connect to a cloud. If this is your only cloud,
# just put this file in ~/.config/openstack/clouds.yaml and tools like
# python-openstackclient will just work with no further config. (You will need
# to add your password to the auth section)
# If you have more than one cloud account, add the cloud entry to the clouds
# section of your existing file and you can refer to them by name with
# OS_CLOUD=openstack or --os-cloud=openstack
clouds:
  kvm:
    auth:
      auth_url: https://kvm.tacc.chameleoncloud.org:5000

      application_credential_id: "c8718aff8a7445d5bc8b1bcbacd89a91"
      application_credential_secret: "4KuiNHWENOg3CKOSFGtR_ip63dx9HJIO44iTfBZMjdugZUMAXqYhxGtmx-6kn0JddKwrC5-8Tl6lUqBpt9glcQ"
      
    region_name: "KVM@TACC"
    interface: "public"
    identity_api_version: 3
    auth_type: "v3applicationcredential"


  chi-tacc:
    auth:
      auth_url: https://chi.

### Applying our Terraform configuration

In [11]:
# runs in Chameleon Jupyter environment
terraform init

Initializing the backend...
Initializing provider plugins...
- Finding terraform-provider-openstack/openstack versions matching "~> 1.51.1"...
- Installing terraform-provider-openstack/openstack v1.51.1...
- Installed terraform-provider-openstack/openstack v1.51.1 (self-signed, key ID 4F80527A391BEFD2)
Partner and community providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://www.terraform.io/docs/cli/plugins/signing.html
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If yo

In [12]:
# runs in Chameleon Jupyter environment
# export TF_VAR_suffix=project-5
# export TF_VAR_key=id_rsa_chameleon_project_5

In [18]:
# runs in Chameleon Jupyter environment
terraform validate

Success! The configuration is valid.



In [23]:
# runs in Chameleon Jupyter environment
terraform plan

data.openstack_networking_subnet_v2.sharednet2_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_3000: Reading...
data.openstack_networking_network_v2.sharednet2: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
openstack_networking_network_v2.private_net: Refreshing state... [id=291457cb-83ed-43f1-b937-c73e2cc6bacd]
data.openstack_networking_secgroup_v2.allow_9090: Read complete after 1s [id=d6736d54-d9a4-431c-a5b5-4d29dbcdee03]
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Read complete after 1s [id=9806d71f-f0bd-4c5d-8c48-f32db3094575]
data.openstack_networking_secgroup_v2.allow_808

Look for the Floating IP that associated to KVM node1, Save for later use

In [24]:
# runs in Chameleon Jupyter environment
terraform apply -auto-approve

data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_3000: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_network_v2.sharednet2: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_subnet_v2.sharednet2_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
openstack_networking_network_v2.private_net: Refreshing state... [id=291457cb-83ed-43f1-b937-c73e2cc6bacd]
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 0s [id=ad2912c0-caa9-49e0-b431-b9b3953b91b3]
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 0s [id=47860acf-9269-4d26-84b1-363824f7e0a2]
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_http